In [1]:
import numpy as np
import pandas as pd
import json
import math
import matplotlib.pyplot as plt
import seaborn as sns
from keplergl import KeplerGl
from datetime import datetime
import time
from dksr_lib import dksr
import requests
import altair as alt
import os
from geojson import LineString as geoLS
from geojson import MultiLineString
import h3
from shapely.geometry import LineString, shape, Point
from shapely.ops import unary_union
from shapely import to_geojson
from turfpy.transformation import convex
import networkx as nx
import osmnx as ox
from collections import Counter
import ast
import pytz
%matplotlib inline

In [3]:
def geo_distance(lon1, lat1, lon2, lat2):
    point1, point2 = [lon1, lat1], [lon2, lat2]
    R = 6378.137 # Radius of earth in KM
    dLat = point2[1] * math.pi / 180 - point1[1] * math.pi / 180
    dLon = point2[0] * math.pi / 180 - point1[0] * math.pi / 180
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(point1[1] * math.pi / 180) * math.cos(point2[1] * math.pi / 180) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c
    return d * 1000 # meters

In [4]:
def haversine_distance_to_point(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6378.137
    
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = np.radians(lat1), np.radians(lon1), np.radians(lat2), np.radians(lon2)
    
    # Compute the differences in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Haversine formula
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    
    return distance

In [5]:
def create_subgraph(lat1,lon1):
    distances = haversine_distance_to_point(lat1,lon1,nodes['y'],nodes['x'])
    nodes['point_distance'] = distances
    sG = G
    for i in range(2):
        sG_edges = list(sG.edges())
        max_edge_length = max([sG.get_edge_data(edge[0],edge[1],0)['length'] for edge in sG_edges])/1000
        search_radius = max_edge_length + distances.min() + 2000
        subnodes = list(nodes[nodes['point_distance'] < search_radius].index)
        sG = G.subgraph(subnodes)
    return sG

In [6]:
aperture_size = 8

In [7]:
files_0 = os.listdir('./Umkreis/')
dates = list(set([x.split('_')[0] for x in files_0]))
dates.sort()

date_list = [s for s in files_0 if dates[1] in s]
df = pd.read_csv('Umkreis/' + date_list[1], compression='gzip', header=None, sep=',')
headers = ['session_id', 'speed', 'timestamp', 'latitude', 'longitude', 'direction']
df.columns = headers
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 763518 entries, 0 to 763517
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   session_id  763518 non-null  int64  
 1   speed       763518 non-null  int64  
 2   timestamp   763518 non-null  object 
 3   latitude    763518 non-null  float64
 4   longitude   763518 non-null  float64
 5   direction   763518 non-null  int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 35.0+ MB


In [8]:
#%%time
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%dT%H:%M:%S.%fZ', utc=True)
#germany_timezone = pytz.timezone('Europe/Berlin')
#df['timestamp'] = df['timestamp'].dt.tz_convert('UTC')
df['unix'] = df['timestamp'].astype(np.int64) // 10**9
df.head()

,session_id,speed,timestamp,latitude,longitude,direction,unix
0,27137987,47,2023-05-21 21:59:03+00:00,49.473380,10.991331,222,1684706343
1,27137987,43,2023-05-21 21:59:08+00:00,49.472992,10.990729,223,1684706348
2,27137987,38,2023-05-21 21:59:13+00:00,49.472603,10.990105,224,1684706353
3,27137987,38,2023-05-21 21:59:18+00:00,49.472256,10.989594,224,1684706358
4,27137987,37,2023-05-21 21:59:23+00:00,49.471905,10.989058,225,1684706363


In [92]:
sessions = df['session_id'].unique()
trip_1 = df[df['session_id']==sessions[0]].reset_index(drop=True)

In [9]:
#lng_min, lng_max, lat_min, lat_max = trip_1['longitude'].min(), trip_1['longitude'].max(), trip_1['latitude'].min(), trip_1['latitude'].max()
lng_min, lng_max, lat_min, lat_max = df['longitude'].min(), df['longitude'].max(), df['latitude'].min(), df['latitude'].max()

G = ox.graph_from_bbox(lat_max+0.02, lat_min-0.02, lng_min-0.02, lng_max+0.02, network_type='drive')
nodes = ox.graph_to_gdfs(G, edges=False)
nodes_dict = nodes.to_dict()

In [10]:
G_edges = list(G.edges())
G_shapes = []
G_names = []
for edge in G_edges:
    try:
        name_i = G.get_edge_data(edge[0],edge[1])[0]['name']
        G_names.append(name_i)
    except:
        G_names.append(None)
    try:
        shape_i = G.get_edge_data(edge[0],edge[1])[0]['geometry']
        G_shapes.append(shape_i)
    except:
        start = (nodes_dict['x'][edge[0]],nodes_dict['y'][edge[0]])
        end = (nodes_dict['x'][edge[1]],nodes_dict['y'][edge[1]])
        G_shapes.append(LineString([start,end]))
    
edge_name_dict = dict(zip(G_edges,G_names))
edge_shape_dict = dict(zip(G_edges,G_shapes))

In [11]:
lat_array = np.arange(lat_min-0.02,lat_max+0.02,0.001)#0.00001)
lng_array = np.arange(lng_min-0.02,lng_max+0.02,0.0015)#0.000015)

In [12]:
h3_indexes = set()
for lat in lat_array:
    for lng in lng_array:
        h3_index = h3.geo_to_h3(lat, lng, aperture_size)
        h3_indexes.add(h3_index)

In [13]:
len(h3_indexes)

6151

In [98]:
h3_df = pd.DataFrame({'h3': list(h3_indexes)})
h3_df['lat'] = h3_df['h3'].apply(lambda x: h3.h3_to_geo(x)[0])
h3_df['lng'] = h3_df['h3'].apply(lambda x: h3.h3_to_geo(x)[1])
h3_df.sort_values(['lat','lng'],ignore_index=True,inplace=True)
h3_df.head()

,h3,lat,lng
0,881fabab63fffff,49.427301,11.033362
1,881fab16bdfffff,49.427694,10.975509
2,881fabab15fffff,49.428457,11.079021
3,881fabab61fffff,49.428873,11.021156
4,881fab16abfffff,49.429259,10.963305


In [14]:
edge_grid_df = pd.DataFrame({'h3':[], 'edges':[], 'distances':[]})
h3_test = list(h3_indexes)#[:100]
for cell in h3_test:
    lat = h3.h3_to_geo(cell)[0]
    lng = h3.h3_to_geo(cell)[1]
    sG = create_subgraph(lat,lng)
    cell_children = []
    sublayer_1 = list(h3.h3_to_children(cell))
    for child_i in sublayer_1:
        sublayer_2 = list(h3.h3_to_children(child_i))
        for child_ij in sublayer_2:
            sublayer_3 = list(h3.h3_to_children(child_ij))
            for child_ijk in sublayer_3:
                sublayer_4 = list(h3.h3_to_children(child_ijk))
                #lats = [h3.h3_to_geo(x)[0] for x in sublayer_4]
                #lngs = [h3.h3_to_geo(x)[1] for x in sublayer_4]
                #edge_tuples = ox.nearest_edges(sG,lngs,lats,return_dist=True)
                #layer_4_df = pd.DataFrame({'h3': sublayer_4, 'distance': edge_tuples[1]})
                #sublayer_4 = list(layer_4_df[layer_4_df['distance'] <= 0.001]['h3'])
                for child_ijkl in sublayer_4:
                    sublayer_5 = list(h3.h3_to_children(child_ijkl))
                    cell_children.extend(sublayer_5)
    lats = [h3.h3_to_geo(x)[0] for x in cell_children]
    lngs = [h3.h3_to_geo(x)[1] for x in cell_children]
    edge_tuples = ox.nearest_edges(sG,lngs,lats,return_dist=True)
    df_i = pd.DataFrame({'h3':cell_children, 'edges':edge_tuples[0], 'distances':edge_tuples[1]})
    df_i = df_i[df_i['distances'] <= 0.0005]
    edge_grid_df = pd.concat([edge_grid_df,df_i], axis=0, ignore_index=True)

In [16]:
edge_grid_df['edges'] = edge_grid_df['edges'].apply(lambda x: (x[0],x[1]))

In [18]:
edge_grid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17182091 entries, 0 to 17182090
Data columns (total 3 columns):
 #   Column     Dtype  
---  ------     -----  
 0   h3         object 
 1   edges      object 
 2   distances  float64
dtypes: float64(1), object(2)
memory usage: 393.3+ MB


In [19]:
edge_grid_df.to_csv('edge_lookup.csv')

In [20]:
edge_grid_df.to_json('edge_lookup.json')

In [69]:
#dksr.geo_distance([test_array[0],lng_min],[test_array[1,lng_min]])
print('Länge lng-Gitter:', geo_distance(lng_array[0], lat_min, lng_array[1], lat_min))
print('Länge lat-Gitter:', geo_distance(lng_min,lat_array[0], lng_min, lat_array[1]))

Länge lng-Gitter: 108.55261711820543
Länge lat-Gitter: 111.3194907929834
